In [69]:
!pip install catboost


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
# import warning

In [88]:
df = pd.read_csv('data/stud.csv')

In [89]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [90]:
X = df.drop(columns=['math score'],axis=1)

In [91]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [92]:
y = df['math score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [93]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [94]:
num_feature = X.select_dtypes(exclude ="object").columns
cat_feature = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer, cat_feature),
        ('StandardScaler', numeric_transformer, num_feature),
    ]
)


In [95]:
X = preprocessor.fit_transform(X)

In [96]:
X.shape

(1000, 19)

In [97]:
# seperate database into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [98]:
#create an evaluate function to give all metrics after model training
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [99]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "k-Neighbors": KNeighborsRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
}

model_list = []
r2_list = []

# Iterate through models
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)  # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate train and test datasets
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Print model results
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print("Model performance for training set")
    print("- Root mean squared error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute error: {:.4f}".format(model_train_mae))
    print("- R2 score: {:.4f}".format(model_train_r2))

    print('----------------')
    print("Model performance for test set")
    print("- Root mean squared error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute error: {:.4f}".format(model_test_mae))
    print("- R2 score: {:.4f}".format(model_test_r2))
    
    r2_list.append(model_test_r2)

    print("=" * 35)

Linear Regression
Model performance for training set
- Root mean squared error: 5.3293
- Mean Absolute error: 4.2715
- R2 score: 0.8740
----------------
Model performance for test set
- Root mean squared error: 5.4252
- Mean Absolute error: 4.2222
- R2 score: 0.8790
Lasso
Model performance for training set
- Root mean squared error: 6.5938
- Mean Absolute error: 5.2063
- R2 score: 0.8071
----------------
Model performance for test set
- Root mean squared error: 6.5197
- Mean Absolute error: 5.1579
- R2 score: 0.8253
Ridge
Model performance for training set
- Root mean squared error: 5.3233
- Mean Absolute error: 4.2650
- R2 score: 0.8743
----------------
Model performance for test set
- Root mean squared error: 5.3904
- Mean Absolute error: 4.2111
- R2 score: 0.8806
k-Neighbors
Model performance for training set
- Root mean squared error: 5.7093
- Mean Absolute error: 4.5175
- R2 score: 0.8554
----------------
Model performance for test set
- Root mean squared error: 7.2583
- Mean Abso

In [102]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name','R2_square']).sort_values(by=["R2_square"],ascending=False) 

,Model Name,R2_square
2,Ridge,0.880593
0,Linear Regression,0.879046
4,Random Forest Regressor,0.851109
1,Lasso,0.825320
3,k-Neighbors,0.783497
